In [213]:
import lightning.pytorch as pl
from lightning.pytorch.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_forecasting import TimeSeriesDataSet, TemporalFusionTransformer, Baseline
from pytorch_forecasting.metrics import MAE
import os
import pandas as pd # type: ignore
import matplotlib.pyplot as plt # type: ignore
import numpy as np # type: ignore
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

In [207]:
stream = pd.read_pickle(os.path.join('..','data','processed','stream_processed.pkl'),compression= 'zip')
meteo = pd.read_pickle(os.path.join('..','data','processed','meteo_processed.pkl'),compression = 'zip')
stream

measurement_location,Datum,15202300,15205501,15207507,15210206,15212008,15212700,15213500,15214003,15214604,15216009,15217908,15221009,15228008,15241006,15242304,15243001,15246000,15247002,15247501
0,2014-01-01,14.4,17.2,19.8,3.87,8.66,9.66,0.433,2.35,0.207,1.66,0.921,0.671,2.59,0.253,0.870,1.26,2.35,0.355,0.120
1,2014-01-02,14.4,17.0,19.3,3.78,8.52,9.46,0.435,2.51,0.204,1.62,0.909,0.664,2.57,0.253,0.870,1.23,2.32,0.361,0.131
2,2014-01-03,14.1,16.6,19.3,3.78,8.40,9.42,0.429,2.61,0.201,1.60,0.907,0.641,2.58,0.253,0.871,1.23,2.30,0.362,0.130
3,2014-01-04,14.2,16.6,19.3,3.84,8.57,9.50,0.445,2.49,0.214,1.64,0.935,0.642,2.60,0.256,0.929,1.26,2.44,0.382,0.127
4,2014-01-05,16.4,18.4,20.6,4.36,9.94,10.80,0.545,2.86,0.267,1.80,1.130,0.690,3.10,0.300,1.150,1.61,3.14,0.561,0.175
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3648,2023-12-28,79.7,102.0,120.0,28.50,46.20,57.10,1.900,17.30,0.721,10.30,6.070,3.920,12.20,0.786,3.270,9.05,12.50,2.020,0.741
3649,2023-12-29,71.4,82.5,97.7,26.00,41.90,51.30,1.710,15.60,0.801,9.85,5.480,3.470,10.90,0.653,2.890,7.23,9.97,1.750,0.619
3650,2023-12-30,64.0,71.4,86.3,24.40,38.60,46.90,1.530,14.70,0.835,9.40,4.900,3.100,9.77,0.557,2.550,5.92,8.54,1.470,0.514
3651,2023-12-31,59.1,63.0,77.3,22.20,35.30,42.50,1.410,13.10,0.669,8.83,4.480,2.670,8.92,0.492,2.330,5.40,7.66,1.290,0.458


In [208]:
stream = stream.iloc[:3652]
data = stream.join(meteo).drop(columns = 'Datum')
data['groups'] = 0
data['date'] = data.index
data['date']

0          0
1          1
2          2
3          3
4          4
        ... 
3647    3647
3648    3648
3649    3649
3650    3650
3651    3651
Name: date, Length: 3652, dtype: int64

In [85]:
data['15207507']

0        19.8
1        19.3
2        19.3
3        19.3
4        20.6
        ...  
3647    161.0
3648    120.0
3649     97.7
3650     86.3
3651     77.3
Name: 15207507, Length: 3652, dtype: float64

In [124]:
# class TimeseriesDataset(Dataset):   
#     def __init__(self, X: np.ndarray,y: np.ndarray, seq_len : int, lag : int):
#         super().__init__()
#         self.X = torch.tensor(X).float()
#         self.y = torch.tensor(y).float()
#         self.seq_len = seq_len
#         self.lag = lag

#     def __len__(self):
#         return self.X.__len__() - (self.seq_len)

#     def __getitem__(self, index):
#         return self.X[index:index+self.seq_len], self.X[index+self.seq_len]

In [131]:
# class StreamFlowDataModule(pl.LightningDataModule):
#     def __init__(self, X : np.array, num_workers = 0,seq_len = 364, batch_size=16):
#         super().__init__()
#         self.X = X
#         self.seq_len = seq_len
#         self.batch_size = batch_size
#         self.num_workers = num_workers

#     def prepare_data(self):
#         self.X_val = self.X[0:int(len(self.X)*0.15)]
#         self.X_train = self.X[int(len(self.X)*0.15)+1:int(len(self.X)*0.85)]
#         self.X_test = self.X[int(len(self.X)*0.85)+1:]
        
#     def train_dataloader(self):
#         train_dataset = TimeseriesDataset(self.X_train, 
#                                           seq_len=self.seq_len)
        
#         train_loader = DataLoader(train_dataset,
#                                   batch_size = self.batch_size, 
#                                   shuffle = True, 
#                                   num_workers = self.num_workers)
#         return train_loader
    
#     def val_dataloader(self):
#         val_dataset = TimeseriesDataset(self.X_val, 
#                                         seq_len=self.seq_len)
        
#         val_loader = DataLoader(val_dataset, 
#                                 batch_size = self.batch_size, 
#                                 shuffle = False, 
#                                 num_workers = self.num_workers)
#         return val_loader

In [186]:
cols = data.columns[20:]

In [199]:
a = data[cols].to_numpy()
b = data['15207507'].to_numpy()
print(a.shape,b[:,np.newaxis].shape)
c = np.concatenate((a,b[:,np.newaxis]),axis=1)

(3652, 94) (3652, 1)


In [143]:
# class LSTMModel(pl.LightningModule):
#     def __init__(self, input_size, hidden_size, seq_len ,dropout, num_layers,criterion,learning_rate = 0.0001):
#         super(LSTMModel, self).__init__()
#         self.hidden_size = hidden_size
#         self.input_size  = input_size
#         self.seq_len = seq_len
#         self.dropout  = dropout
#         self.criterion = criterion
#         self.learning_rate = learning_rate

        
#         self.lstm = nn.LSTM(input_size=self.input_size, 
#                             hidden_size=self.hidden_size,
#                             num_layers=num_layers, 
#                             dropout=dropout, 
#                             batch_first=True)
        
#         self.linear = nn.Linear(self.hidden_size, 1)
        
#     def forward(self, x):
#         # lstm_out = (batch_size, seq_len, hidden_size)
#         lstm_out, _ = self.lstm(x.unsqueeze(dim = 2))
#         y_pred = self.linear(lstm_out[:,-1])
#         return y_pred
    
#     def training_step(self, batch, batch_idx):
#         x, y = batch
#         y_hat = self(x)
#         loss = self.criterion(y_hat, y)
#         self.log('train_loss', loss,on_step=True, on_epoch=True, prog_bar=True)
#         return loss

#     def validation_step(self, batch, batch_idx):
#         x, y = batch
#         y_hat = self(x)
#         loss = self.criterion(y_hat, y)
#         self.log('val_loss', loss,on_epoch = True, prog_bar = True)
#         return loss
    
#     def configure_optimizers(self):
#         return torch.optim.Adam(self.parameters(), lr=self.learning_rate)

In [ ]:
# trainer = pl.Trainer(
#         max_epochs=7,
#         accelerator = 'cpu',
#         precision=16,
#         fast_dev_run= False
#     )

#     # Example usage
# input_size = 1  # Assuming univariate time series data
# hidden_size = 16
# output_size = 1  # Assuming predicting one step ahead
# batch_size = 64
# seq_len = 364
# dropout = 0
# num_layers = 32
# criterion = nn.L1Loss()

# model = LSTMModel(input_size, hidden_size, dropout= dropout, seq_len=seq_len,num_layers=num_layers,criterion = criterion)

# dm = StreamFlowDataModule(X = data.to_numpy(),batch_size=batch_size)
# # dm.prepare_data()
# # first_batch = next(iter(dm.train_dataloader()))[0].unsqueeze(dim = 2)
# # print(first_batch)
# trainer.fit(model, datamodule= dm)

In [211]:
max_pred_len = 6
training_cutoff = data["date"].max() - max_pred_len

training = TimeSeriesDataSet(
    data.iloc[:training_cutoff],
    target='15207507',
    group_ids=['groups'],
    time_idx='date',
    max_encoder_length=364,
    min_prediction_length=1,
    max_prediction_length=max_pred_len,
    time_varying_unknown_reals=['15207507'],
    time_varying_known_reals=data.columns[20:-1].values.tolist()
)
validation = TimeSeriesDataSet.from_dataset(training, data, predict=True, stop_randomization=True)


In [212]:
# convert the dataset to a dataloader
train_dataloader = training.to_dataloader(train=True, batch_size=32, num_workers=2)
val_dataloader = validation.to_dataloader(train=False, batch_size=32, num_workers=2)



In [108]:
# baseline_predictions = Baseline().predict(val_dataloader, return_y=True)
# MAE()(baseline_predictions.output, baseline_predictions.y)

/Users/lemarx/anaconda3/envs/Climate_Data/lib/python3.10/site-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
/Users/lemarx/anaconda3/envs/Climate_Data/lib/python3.10/site-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/lemarx/anaconda3/envs/Climate_Data/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/logger_connector/logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` p

tensor(192.1167, device='mps:0')

In [216]:
pl.seed_everything(42)
trainer = pl.Trainer(
    accelerator="cpu",
    max_epochs=3,
    # clipping gradients is a hyperparameter and important to prevent divergance
    # of the gradient for recurrent neural networks
    gradient_clip_val=0.1,
)


tft = TemporalFusionTransformer.from_dataset(
    training,
    # not meaningful for finding the learning rate but otherwise very important
    learning_rate=0.03,
    hidden_size=64,  # most important hyperparameter apart from learning rate
    # number of attention heads. Set to up to 4 for large datasets
    attention_head_size=1,
    dropout=0.2,  # between 0.1 and 0.3 are good values
    hidden_continuous_size=64,  # set to <= hidden_size
    loss=MAE(),
    optimizer="Ranger"
    # reduce learning rate if no improvement in validation loss after x epochs
    # reduce_on_plateau_patience=1000,
)

Seed set to 42
GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/lemarx/anaconda3/envs/Climate_Data/lib/python3.10/site-packages/lightning/pytorch/trainer/setup.py:187: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
/Users/lemarx/anaconda3/envs/Climate_Data/lib/python3.10/site-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
/Users/lemarx/anaconda3/envs/Climate_Data/lib/python3.10/site-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.


In [217]:
# fit network
trainer.fit(
    tft,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
)


   | Name                               | Type                            | Params
----------------------------------------------------------------------------------------
0  | loss                               | MAE                             | 0     
1  | logging_metrics                    | ModuleList                      | 0     
2  | input_embeddings                   | MultiEmbedding                  | 0     
3  | prescalers                         | ModuleDict                      | 12.3 K
4  | static_variable_selection          | VariableSelectionNetwork        | 0     
5  | encoder_variable_selection         | VariableSelectionNetwork        | 2.0 M 
6  | decoder_variable_selection         | VariableSelectionNetwork        | 2.0 M 
7  | static_context_variable_selection  | GatedResidualNetwork            | 16.8 K
8  | static_context_initial_hidden_lstm | GatedResidualNetwork            | 16.8 K
9  | static_context_initial_cell_lstm   | GatedResidualNetwork            | 16.8

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/Users/lemarx/anaconda3/envs/Climate_Data/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:436: Consider setting `persistent_workers=True` in 'val_dataloader' to speed up the dataloader worker initialization.


/Users/lemarx/anaconda3/envs/Climate_Data/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:436: Consider setting `persistent_workers=True` in 'train_dataloader' to speed up the dataloader worker initialization.


Epoch 0:  14%|█▎        | 14/102 [00:34<03:35,  0.41it/s, v_num=3, train_loss_step=11.60]

/Users/lemarx/anaconda3/envs/Climate_Data/lib/python3.10/site-packages/lightning/pytorch/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...
